In [346]:
"""
MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 20 YEARS OF DATA

First,  we use the backtesting library to find the best crossover strategy using 20 years of data.
Next, we confirm the results using our own backtesting and metrics. 

The backtesting library choose short = 215, long = 225. It tested 2556 combinations using short = 5 to 365 with a step of 5
and long = 5 to 365 with a step of 5. 

"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

%matplotlib inline

In [347]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [348]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-06-26,306.1600,306.39,299.4200,300.05,300.05,127811745.0,0.0,1.0
2020-06-25,303.4700,307.64,301.2800,307.35,307.35,88966079.0,0.0,1.0
2020-06-24,309.8400,310.51,302.1000,304.09,304.09,132067392.0,0.0,1.0
2020-06-23,313.4801,314.50,311.6101,312.05,312.05,68066900.0,0.0,1.0
2020-06-22,307.9900,311.05,306.7500,310.62,310.62,74007212.0,0.0,1.0


In [367]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



In [350]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.sort_index(ascending=True, inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2000-06-26,145.3750,146.2500,144.8750,99.9424,5201300.0
2000-06-27,145.9843,146.7187,145.0156,99.2056,4159500.0
2000-06-28,145.6250,146.9843,145.3125,99.4832,5347700.0
2000-06-29,144.7500,145.7500,143.5156,98.5435,6345700.0
2000-06-30,143.9375,145.5937,143.8906,99.2910,7420200.0


In [344]:
#Function to do a backtest using Bactesting library. This function will find the best SMA cross strategy.

def backtest():
    class SmaCross(Strategy):
        
        n1=1
        n2=365
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=10000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(5, 365, 5),
                    n2=range(5, 365, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:28: UserWarning:

Searching best of 2556 configurations.



Start                         2000-06-26 00:00:00
End                           2020-06-26 00:00:00
Duration                       7305 days 00:00:00
Exposure [%]                              87.7892
Equity Final [$]                          54961.9
Equity Peak [$]                           61325.8
Return [%]                                449.619
Buy & Hold Return [%]                     200.223
Max. Drawdown [%]                        -52.4712
Avg. Drawdown [%]                        -4.34016
Max. Drawdown Duration         1348 days 00:00:00
Avg. Drawdown Duration           58 days 00:00:00
# Trades                                       30
Win Rate [%]                              66.6667
Best Trade [%]                            66.1451
Worst Trade [%]                          -12.5163
Avg. Trade [%]                             7.2533
Max. Trade Duration            1294 days 00:00:00
Avg. Trade Duration             214 days 00:00:00
Expectancy [%]                            9.09299


In [351]:
# To confirm the above results (short = 215, long = 225) we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2000-06-26,99.9424
2000-06-27,99.2056
2000-06-28,99.4832
2000-06-29,98.5435
2000-06-30,99.2910


In [352]:
# Set the short window and long windows
short_window = 215
long_window = 225

# Set the `date` column as the index
#signals_df.set_index("date", drop=True)

# Generate the short and long moving averages (215 and 225 days, respectively)
signals_df["SMA215"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA225"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA215"][short_window:] > signals_df["SMA225"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA215,SMA225,Signal,Entry/Exit
date,,,,,
2000-06-26,99.9424,NaN,NaN,0.0,NaN
2000-06-27,99.2056,NaN,NaN,0.0,0.0
2000-06-28,99.4832,NaN,NaN,0.0,0.0
2000-06-29,98.5435,NaN,NaN,0.0,0.0
2000-06-30,99.2910,NaN,NaN,0.0,0.0
2000-07-03,100.6579,NaN,NaN,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0


In [353]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA215', 'SMA225']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [354]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 1300

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

#Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(10)

,Close,SMA215,SMA225,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2000-06-26,99.9424,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-27,99.2056,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2000-06-28,99.4832,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-06-29,98.5435,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-06-30,99.2910,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-07-03,100.6579,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0


In [355]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [356]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [357]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df



,Backtest
Annual Return,NaN
Cumulative Returns,3.16391
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [358]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0785511
Cumulative Returns,3.16391
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [359]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0785511
Cumulative Returns,3.16391
Annual Volatility,0.118857
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [360]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0785511
Cumulative Returns,3.16391
Annual Volatility,0.118857
Sharpe Ratio,0.660885
Sortino Ratio,NaN


In [361]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,0.0785511
Cumulative Returns,3.16391
Annual Volatility,0.118857
Sharpe Ratio,0.660885
Sortino Ratio,NaN


In [362]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,0.0785511
Cumulative Returns,3.16391
Annual Volatility,0.118857
Sharpe Ratio,0.660885
Sortino Ratio,0.919303


In [363]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [364]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2002-02-04,2002-03-04,1300.0,76.5107,80.6182,99463.91,104803.66,5339.75
1,SPY,2002-07-29,2002-08-23,1300.0,62.8969,66.2810,81765.97,86165.30,4399.33
2,SPY,2003-05-30,2003-06-26,1300.0,68.8334,70.4010,89483.42,91521.30,2037.88
3,SPY,2003-07-18,2004-11-26,1300.0,70.9069,86.2802,92178.97,112164.26,19985.29
4,SPY,2004-12-15,2004-12-17,1300.0,88.1247,87.4889,114562.11,113735.57,-826.54
5,SPY,2005-01-19,2008-03-06,1300.0,86.5952,101.3263,112573.76,131724.19,19150.43
6,SPY,2009-09-28,2010-11-19,1300.0,85.6597,98.8290,111357.61,128477.70,17120.09
7,SPY,2010-11-29,2011-02-09,1300.0,97.9006,109.2424,127270.78,142015.12,14744.34
8,SPY,2011-03-14,2011-11-25,1300.0,107.4089,97.4773,139631.57,126720.49,-12911.08
9,SPY,2012-04-23,2012-05-09,1300.0,115.8423,114.9531,150594.99,149439.03,-1155.96


In [341]:
#Create plots

price_df = signals_df[['Close', 'SMA215', 'SMA225']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [342]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [343]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [307]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [308]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)